# 📚 Mobile VLA + Kosmos 2B 학습 노트북

## 🎯 목표
- **Mobile Robot Navigation**: 장애물 회피 컵 추적 태스크
- **Kosmos 2B 활용**: RoboVLMs 방식으로 VLM 백본 사용
- **3D 액션 학습**: [linear_x, linear_y, angular_z] 모바일 로봇 액션

## 📊 데이터셋 정보
- **72개 에피소드** (1 episode = 1 task sample)
- **8개 시나리오**: 1box/2box × left/right × vertical/horizontal
- **18 프레임 시퀀스**: mobile_vla_data_collector.py 표준
- **영어 명령어**: "Navigate around obstacles to track the target cup"


In [1]:
# 📦 필수 라이브러리 임포트
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
import logging
from torch.utils.data import DataLoader
import importlib.util
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 플롯 스타일 설정
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(levelname)s:%(name)s:%(message)s')
logger = logging.getLogger(__name__)

print("🚀 Mobile VLA + Kosmos 학습 환경 준비 완료!")
print(f"📱 PyTorch: {torch.__version__}")
print(f"🔥 CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")


🚀 Mobile VLA + Kosmos 학습 환경 준비 완료!
📱 PyTorch: 2.3.0+cu121
🔥 CUDA 사용 가능: True
🎮 GPU: NVIDIA RTX A5000


## 🔧 환경 설정 및 데이터셋 로드


In [3]:
# 🗂️ 경로 설정
ROOT_DIR = Path.cwd()
DATA_DIR = Path("/home/billy/25-1kp/vla/ROS_action/mobile_vla_dataset/")
SAVE_DIR = ROOT_DIR / "experiments" / f"training_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 작업 디렉토리: {ROOT_DIR}")
print(f"📊 데이터 디렉토리: {DATA_DIR}")
print(f"💾 저장 디렉토리: {SAVE_DIR}")

# 🔄 동적 임포트 (Jupyter 환경에서 모듈 로드)
def load_class_from_file(module_name: str, file_path: str, class_name: str):
    """파일에서 클래스를 동적으로 로드"""
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return getattr(module, class_name)

# 🤖 Kosmos 프로세서 초기화
from transformers import AutoProcessor
print("🤖 Kosmos 프로세서 로드 중...")
kosmos_processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")
print("✅ Kosmos 프로세서 로드 완료!")

# 데이터셋 클래스 로드
MobileVLAToRoboVLMsAdapter = load_class_from_file(
    'robovlms_adapter',
    str(ROOT_DIR / 'data' / 'robovlms_adapter.py'),
    'MobileVLAToRoboVLMsAdapter'
)

# 데이터셋 초기화 (올바른 파라미터 사용)
dataset = MobileVLAToRoboVLMsAdapter(
    data_dir=str(DATA_DIR),
    sequence_length=18,
    scenario_filter=None,  # 모든 시나리오 사용 (unknown 제외는 내부적으로 처리됨)
    image_processor=kosmos_processor
)

print(f"\n📊 데이터셋 정보:")
print(f"   총 에피소드: {len(dataset)}개")
print(f"   시나리오 분포: {dataset.get_scenario_statistics()}")

# 첫 번째 샘플 확인
if len(dataset) > 0:
    sample = dataset[0]
    print(f"\n🔍 샘플 데이터 형태:")
    for key, value in sample.items():
        if torch.is_tensor(value):
            print(f"   {key}: {value.shape} ({value.dtype})")
        else:
            print(f"   {key}: {value}")

📁 작업 디렉토리: /home/billy/25-1kp/vla/Robo+/Mobile_VLA
📊 데이터 디렉토리: /home/billy/25-1kp/vla/ROS_action/mobile_vla_dataset
💾 저장 디렉토리: /home/billy/25-1kp/vla/Robo+/Mobile_VLA/experiments/training_20250818_235321
🤖 Kosmos 프로세서 로드 중...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:mobile_dataset:📁 Mobile VLA Dataset 로드 완료!
INFO:mobile_dataset:📊 총 72개 에피소드, 1296개 프레임
INFO:mobile_dataset:🎯 시나리오 분포: {'2box_vert_left': 10, '2box_hori_right': 6, '1box_vert_left': 10, '1box_hori_right': 10, '2box_vert_right': 10, '1box_hori_left': 10, '1box_vert_right': 10, '2box_hori_left': 6}
INFO:mobile_dataset:🎯 18프레임 에피소드: 8개 (표준 길이)


✅ Kosmos 프로세서 로드 완료!

📊 데이터셋 정보:
   총 에피소드: 72개
   시나리오 분포: {'2box_vert_left': 10, '2box_hori_right': 6, '1box_vert_left': 10, '1box_hori_right': 10, '2box_vert_right': 10, '1box_hori_left': 10, '1box_vert_right': 10, '2box_hori_left': 6}

🔍 샘플 데이터 형태:
   vision_x: torch.Size([1, 18, 3, 224, 224]) (torch.float32)
   task_description: Navigate around the two box obstacles by going left to track the target cup
   scenario: 2box_vert_left
   episode_name: episode_20250815_122923_2box_vert_left_core_medium
   num_frames: 18
   mobile_actions: torch.Size([1, 18, 3]) (torch.float32)
   mobile_events: torch.Size([1, 18]) (torch.int64)


## 🤖 트레이너 및 로스 트래커 설정


In [4]:
# 트레이너 클래스 로드
MobileKosmosTrainer = load_class_from_file(
    'kosmos_trainer',
    str(ROOT_DIR / 'training' / 'kosmos_trainer.py'),
    'MobileKosmosTrainer'
)

# 🔧 학습 설정
training_config = {
    'kosmos_model_name': "microsoft/kosmos-2-patch14-224",
    'freeze_kosmos': True,
    'batch_size': 1,  # GPU 메모리에 따라 조정
    'sequence_length': 18,
    'policy_head_hidden_size': 768,
    'use_policy_lstm': True,
    'policy_lstm_layers': 2,
    'dropout': 0.1,
    'learning_rate': 1e-4,
    'weight_decay': 0.01,
    'max_epochs': 10,
    'action_loss_weight': 1.0,
    'event_loss_weight': 0.5,
    'warmup_steps': 5,
    'log_interval': 5  # 몇 스텝마다 로그 출력
}

print("⚙️ 학습 설정:")
for key, value in training_config.items():
    print(f"   {key}: {value}")

# 트레이너 초기화
trainer = MobileKosmosTrainer(training_config)
print("\n🤖 Mobile Kosmos Trainer 초기화 완료!")

# 데이터로더 생성
dataloader = DataLoader(
    dataset, 
    batch_size=training_config['batch_size'], 
    shuffle=True, 
    num_workers=2, 
    drop_last=True,
    pin_memory=torch.cuda.is_available()
)

print(f"\n📦 데이터로더 정보:")
print(f"   배치 크기: {training_config['batch_size']}")
print(f"   총 배치 수: {len(dataloader)}")
print(f"   워커 수: 2")


⚙️ 학습 설정:
   kosmos_model_name: microsoft/kosmos-2-patch14-224
   freeze_kosmos: True
   batch_size: 1
   sequence_length: 18
   policy_head_hidden_size: 768
   use_policy_lstm: True
   policy_lstm_layers: 2
   dropout: 0.1
   learning_rate: 0.0001
   weight_decay: 0.01
   max_epochs: 10
   action_loss_weight: 1.0
   event_loss_weight: 0.5
   warmup_steps: 5
   log_interval: 5


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:kosmos_trainer:🔒 Kosmos 가중치 고정됨
INFO:models.policy_heads.mobile_policy_head:🎯 Mobile Policy Head 초기화 완료
INFO:models.policy_heads.mobile_policy_head:   액션 차원: 3, 이벤트 타입: 3
INFO:models.policy_heads.mobile_policy_head:   LSTM 사용: True, Hidden: 768
INFO:kosmos_trainer:🤖 Mobile Kosmos Model 초기화 완료
INFO:kosmos_trainer:   Kosmos Hidden: 2048, Mobile Hidden: 768
INFO:kosmos_trainer:🤖 Mobile Kosmos Trainer 초기화 완료
INFO:kosmos_trainer:   학습 가능 파라미터: 11,978,249개



🤖 Mobile Kosmos Trainer 초기화 완료!

📦 데이터로더 정보:
   배치 크기: 1
   총 배치 수: 72
   워커 수: 2


In [5]:
# 📊 로스 트래커 클래스
class LossTracker:
    """학습 로스를 추적하고 시각화하는 클래스"""
    
    def __init__(self, save_dir: Path):
        self.save_dir = save_dir
        self.losses = {
            'total_loss': [],
            'action_loss': [],
            'event_loss': [],
            'learning_rates': [],
            'steps': [],
            'scenarios': []
        }
        
    def update(self, step: int, losses: dict, lr: float, scenario: str):
        """로스 업데이트"""
        self.losses['steps'].append(step)
        self.losses['total_loss'].append(losses['total_loss'])
        self.losses['action_loss'].append(losses['action_loss'])
        self.losses['event_loss'].append(losses['event_loss'])
        self.losses['learning_rates'].append(lr)
        self.losses['scenarios'].append(scenario)
        
    def plot_losses(self, window_size: int = 10):
        """로스 시각화"""
        if len(self.losses['steps']) == 0:
            print("📊 아직 로스 데이터가 없습니다.")
            return
            
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        steps = self.losses['steps']
        
        # 총 로스
        axes[0, 0].plot(steps, self.losses['total_loss'], 'b-', alpha=0.7, label='Total Loss')
        if len(steps) > window_size:
            # 이동평균 추가
            moving_avg = np.convolve(self.losses['total_loss'], 
                                   np.ones(window_size)/window_size, mode='valid')
            axes[0, 0].plot(steps[window_size-1:], moving_avg, 'r-', linewidth=2, 
                          label=f'Moving Avg ({window_size})')
        axes[0, 0].set_title('📈 Total Loss', fontweight='bold')
        axes[0, 0].set_xlabel('Steps')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # 액션 로스
        axes[0, 1].plot(steps, self.losses['action_loss'], 'g-', alpha=0.7, label='Action Loss')
        axes[0, 1].set_title('🎯 Action Loss (3D Mobile)', fontweight='bold')
        axes[0, 1].set_xlabel('Steps')
        axes[0, 1].set_ylabel('MSE Loss')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # 이벤트 로스
        axes[1, 0].plot(steps, self.losses['event_loss'], 'orange', alpha=0.7, label='Event Loss')
        axes[1, 0].set_title('🎭 Event Loss', fontweight='bold')
        axes[1, 0].set_xlabel('Steps')
        axes[1, 0].set_ylabel('CrossEntropy Loss')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # 학습률
        axes[1, 1].plot(steps, self.losses['learning_rates'], 'purple', alpha=0.7)
        axes[1, 1].set_title('📉 Learning Rate', fontweight='bold')
        axes[1, 1].set_xlabel('Steps')
        axes[1, 1].set_ylabel('LR')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(self.save_dir / 'training_losses.png', dpi=300, bbox_inches='tight')
        plt.show()

# 로스 트래커 초기화
loss_tracker = LossTracker(SAVE_DIR)
print("📊 로스 트래커 초기화 완료!")


📊 로스 트래커 초기화 완료!
